In [30]:
import requests 
import csv
###Look for sequence ID based on FASTA seqences

#    my_sequence = """
#    >example_sequence
#    MADEEKLPPGWEKRMSRSSGRVYYFNHITNASQWERPSGNSSSGSRGPTAAWGSSAASS
#    """
def get_STRING_id(sequence,species):
    blast_url = "https://string-db.org/api/tsv-no-header/resolve"
    blast_params = {
        "sequence": sequence,  # 查询的蛋白质序列
        "species": species,  # NCBI 物种标识符（9606 代表人类）
        "caller_identity": "cuhk"  # 替换为你的应用名称或标识符
    }
    response = requests.post(blast_url, data=blast_params)
    if response.status_code == 200:
        matched_proteins = []
        for line in response.text.strip().split("\n"):
            print(line)
            columns = line.split("\t")
            if len(columns) > 1:
                protein_id = columns[0]  
                matched_proteins.append(protein_id)
        if matched_proteins:
            print("找到的 STRING 蛋白质 ID 列表：", matched_proteins)
        else:
            print("未找到匹配的蛋白质")
    else:
        print(f"BLAST 查询失败，状态码: {response.status_code}")
s = """
>AEI93967.1 putative carbohydrate kinase [Roseobacter litoralis Och 149]
MAISILNRSNLPLDLLQKKQGHKFDHGHALVLSGGVGRTGAARLAAMAALRIGAGVVSIGTPPAAISEVA
AHISALMVRKIKTPHDLEIALSDKRISALCVGPGFGTSPAQIGLIEEILAAKRATVLDADAITLLAMNRE
LCTLLHPACVLTPHAGEFARLFPDTSAEAFAASIDKRAEVTINAAHDMGCIVIAKGEDTLVADHNGHVYL
HQATGARAAPWLATAGSGDVLAGFITGLMARGFTPLEAAKVAVWLHVECAITFGPGLIAEDLPAVLPKVL
RKLLV
"""
get_STRING_id(s,"auto-detect")

4932.YOR137C	4932	Saccharomyces cerevisiae	SIA1	Protein of unassigned function; involved in activation of the Pma1p plasma membrane H+-ATPase by glucose; contains peptide signal for membrane localization.
找到的 STRING 蛋白质 ID 列表： ['4932.YOR137C']


In [31]:
### Look for interaction based on sequence ID

def get_interaction_info(output_file,gene_list,species):
    output_format=output_file.split(".")[-1]
    string_api_url = "https://version-11-5.string-db.org/api"
    method = "network"
    request_url = "/".join([string_api_url, output_format, method])
    params = {  
        "identifiers" : "%0d".join(gene_list), 
        "species" : species, 
        "caller_identity" : "cuhk" 
    }
    
    response = requests.post(request_url, data=params)
    if response.status_code == 200:
        with open(output_file, "w") as file:
            file.write(response.text)
    else:
        print(f"请求失败，状态码: {response.status_code}")

get_interaction_info("/home-user/ymchen/4932.YOR137C_download.tsv",["4932.YOR137C"],"auto-detect")

In [36]:
###extract interaction information from tsv file 
###need list(set())
import csv
def get_connectivity(tsv_file,target_seq):
    count = 0
    unique_rows = set()
    with open(tsv_file, mode='r', newline='', encoding='utf-8') as file:
        tsv_reader = csv.reader(file, delimiter='\t')
        for row in tsv_reader:
            unique_rows.add(tuple(row))
    for row in unique_rows:
        if target_seq in row:
            count += 1
    
    print(f"去重后包含目标值 '{target_seq}' 的行数为: {count}")
get_connectivity("/home-user/ymchen/4932.YOR137C_download.tsv","4932.YGL008C")

去重后包含目标值 '4932.YGL008C' 的行数为: 3


In [ ]:
### carry out connectivity calculation of ortholog for pseudogenes